Reading Data

In [1]:
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from spellchecker import SpellChecker

In [3]:
reviews = pd.read_csv('train_data.csv', quoting=1, escapechar='\\', on_bad_lines='skip')

<ipython-input-3-16778e4a6689>:1: DtypeWarning: Columns (0,1,2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv('train_data.csv', quoting=1, escapechar='\\', on_bad_lines='skip')


In [4]:
mapping_data_head = reviews.head()
mapping_data_head

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400.0
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200.0
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000.0
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600.0
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600.0


In [5]:
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\W+', ' ', text)
        return text
    return ""

In [6]:
reviews['processed_text'] = reviews['reviewText'].apply(preprocess_text)
reviews['processed_text']

,processed_text
0,i have an older urc wr7 remote and thought thi...
1,first time i ve ever had a remote that needed ...
2,got them and only 2 of them worked company cal...
3,i got tired of the remote being on the wrong s...
4,after purchasing cheap cords from another webs...
...,...
462419,div id video block r13gnqiufepjci class a sec...
462420,fit my wifes ipad mini 2 perfectly ive had the...
462421,i bought this in december 2017 and the rotatin...
462422,i bought this case as a replacement for one i ...


In [7]:
similar_words = ["warranty", "guarantee"]

In [8]:
spell = SpellChecker()
typos = [spell.unknown(similar_words)]
typos

[set()]

In [9]:
typos = list(spell.unknown(similar_words))

In [10]:
def is_relevant(text, keywords):
    return any(word in text for word in keywords)

In [11]:
keywords = similar_words + typos
reviews['is_relevant'] = reviews['processed_text'].apply(lambda x: is_relevant(x, keywords))

In [12]:
filtered_reviews = reviews[reviews['is_relevant']]

In [21]:
reviews['overall'] = pd.to_numeric(reviews['overall'], errors='coerce')

In [22]:
reviews = reviews.dropna(subset=['overall'])

In [24]:
filtered_reviews = reviews[reviews['is_relevant']]
average_rating = filtered_reviews.groupby('asin')['overall'].mean()
average_rating

,overall
asin,
B000001OM4,4.0000
B00000J4EY,3.0000
B00000K2YR,1.0000
B00001P4XA,5.0000
B00001P4ZH,4.3125
...,...
B00YJRAVCW,1.0000
B00YJS4RLM,1.0000
B00YM3SEJK,1.0000
